In [1]:
import ee
import geemap

In [2]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
#Create a point for EC Flux Tower
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
#Create a 0.5x0.5 deg region centered over the EC Flux Tower
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [3]:
#Add the rectangle to the map
Map.addLayer(brma2_region, {}, 'Manauas Region')

In [5]:
#Load Sentinel-2 data and filter out cloud cover for tiles with less than 20% coverage
#Note: there is also a cloud masking function that I haven't tried. This was just easy.
sentinel2 = (ee.ImageCollection("COPERNICUS/S2") 
    .filterDate('2015-01-01', '2020-01-01') 
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
#    .filterBounds(brma2))
    .map(lambda image : image.clip(brma2_region)))

#Create some parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'max': 2000
}

Map.addLayer(sentinel2, vis_params, 'Raw Sentinel')

In [6]:
# Add function to calculate NDRE
def addNDRE(image):
    return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE'))

# Map addNDRE function to all sentinel2 images
sentinel2 = sentinel2.map(addNDRE)

# extract just the NDRE band from all images
sentinel2_ndre= sentinel2.select('NDRE')

# take median over the image stack to get a single NDRE image
sentinel2_ndre_median = sentinel2_ndre.median()

In [7]:
# visualize the NDRE
ndreParams={'min': 0.0,
            'max': 1.0,
            'palette' : ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
           }

Map.addLayer(sentinel2_ndre_median, ndreParams, 'NDRE Image')

In [8]:
#Add the Manaus location to the map
Map.addLayer(brma2_point, {}, 'Manauas EC Tower')